In [24]:
from bs4 import BeautifulSoup
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.common.by import By
import re
import requests
import pymysql


In [34]:
# 네이버 증권에 접속 -> 해당 기업의 산업 코드 얻기
def get_category(code):
    try:
        url = f"https://finance.naver.com/item/main.naver?code={code}"
        res = requests.get(url)
        soup = BeautifulSoup(res.text, "html.parser")
        category = soup.select_one('div.trade_compare>h4>em>a').text
        return category

    except:
        print("알 수 없는 에러 발생")
        return
    
# get_category("051910")

In [35]:
#pymysql 커서를 띄워 놓음 - 닫기는 db.close()
db = pymysql.connect(host='localhost', user = 'root', passwd = '0000', db = 'project02', charset = 'utf8')

In [46]:
from pymysql import IntegrityError

get_name = re.compile(".+(?=\()")
get_code = re.compile("([0-9]{6})")

with db.cursor() as curs: # 나중에 db에 데이터 저장할 때 밑에 db.commit(), db.close() 달아주자 / commit()없이는 저장안됨
    with webdriver.Chrome('./chromedriver.exe') as driver:
        driver.implicitly_wait(2)
        for i in range(1, 3):
            driver.get(f'http://hkconsensus.hankyung.com/apps.analysis/analysis.list?sdate=2022-05-01&edate=2022-08-03&now_page={i}&search_value=&report_type=UP&pagenum=80&search_text=&business_code=')

            contents = driver.find_elements(By.CSS_SELECTOR, 'table>tbody tr')
            
            for j in contents:
                try:
                    date = j.find_element(By.CSS_SELECTOR, 'td.txt_number').text
                    report = j.find_element(By.CSS_SELECTOR, 'td.text_l').text
                    name = get_name.findall(report)[0]
                    code = get_code.findall(report)[0]
                    category = get_category(str(code))
                    made_by = j.find_element(By.CSS_SELECTOR, 'td:nth-child(5)').text

                    
                    print(name, code, date, category, made_by)
                    # pymysql의 쿼리를 넣는 방식은 이렇다
                
                    query1 = "INSERT INTO companies(company, code, category) VALUES(%s, %s, %s)";
                    query2 = "INSERT INTO reports(company, date, written_by) VALUES(%s, %s, %s);"
                    
                    curs.execute(query1, (name, code, category))
                    curs.execute(query2, (name, date, made_by))
                    

                    # print(curs.execute("SELECT * FROM companies JOIN reports ON companies.company = reports.company"))
                except :
                    print("몬가 실행되지 않음")
    
    query3 = "SELECT * FROM companies JOIN reports ON companies.company = reports.company"
    curs.execute(query3)
    result=curs.fetchall() # 커서는 쿼리를 실행한 마지막 위치에 있는 것 같다.
    for i in result:
        print(i)
        
# db.commit() # 변경 내역 저장
# db.close() # 열린 db 닫기

C:\Users\dowra\AppData\Local\Temp\ipykernel_5096\913187193.py:8: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  with webdriver.Chrome('./chromedriver.exe') as driver:


제이브이엠 054950 2022-08-03 건강관리장비와용품 한화투자증권
몬가 실행되지 않음
한국타이어앤테크놀로지 161390 2022-08-03 자동차부품 대신증권
몬가 실행되지 않음
한국타이어앤테크놀로지 161390 2022-08-03 자동차부품 메리츠증권
몬가 실행되지 않음
F&F 383220 2022-08-02 섬유,의류,신발,호화품 대신증권
몬가 실행되지 않음
만도 204320 2022-08-02 자동차부품 메리츠증권
몬가 실행되지 않음
F&F 383220 2022-08-02 섬유,의류,신발,호화품 메리츠증권
몬가 실행되지 않음
한화에어로스페이스 012450 2022-08-01 우주항공과국방 유진투자증권
몬가 실행되지 않음
현대중공업 329180 2022-08-01 조선 메리츠증권
몬가 실행되지 않음
현대미포조선 010620 2022-08-01 조선 메리츠증권
몬가 실행되지 않음
만도 204320 2022-08-01 자동차부품 메리츠증권
몬가 실행되지 않음
대웅제약 069620 2022-07-29 제약 SK증권
몬가 실행되지 않음
한화솔루션 009830 2022-07-29 화학 SK증권
몬가 실행되지 않음
고려아연 010130 2022-07-29 비철금속 하이투자증권
몬가 실행되지 않음
제이브이엠 054950 2022-07-29 건강관리장비와용품 이베스트증권
몬가 실행되지 않음
대웅제약 069620 2022-07-29 제약 이베스트증권
몬가 실행되지 않음
SK바이오사이언스 302440 2022-07-29 제약 이베스트증권
몬가 실행되지 않음
LG이노텍 011070 2022-07-29 전자장비와기기 이베스트증권
몬가 실행되지 않음
한화솔루션 009830 2022-07-29 화학 유진투자증권
몬가 실행되지 않음
제일기획 030000 2022-07-29 광고 유진투자증권
몬가 실행되지 않음
삼성전자 005930 2022-07-29 반도체와반도체장비 유진투자증권
몬가 실행되지 않음
LG생활건강 051900 2022-07-29 화장품 유안타증권
몬가 실행되지 